In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Helper functions

def CopyPointCloud(input_pcd: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
    copy_point_cloud = o3d.geometry.PointCloud()

    # copying contents
    copy_point_cloud.points = input_pcd.points
    copy_point_cloud.colors = input_pcd.colors

    return copy_point_cloud

def WritePointCloud(file_path: str, pcd: o3d.geometry.PointCloud) -> None: 
    o3d.io.write_point_cloud(file_path, pcd)

def ReadCleanPointCloud(file_path: str, nb_neighbors=20, std_ratio=2.0) -> o3d.geometry.PointCloud:
    pcd = o3d.io.read_point_cloud(file_path)
    pcd = pcd.remove_duplicated_points()
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors,
                                                    std_ratio)
    
    return pcd

def RenderPointCloud(pcd: o3d.geometry.PointCloud) -> None:
    o3d.visualization.draw_geometries([pcd])

# def NormalizedPointCloud(pcd: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
#     # Load your point cloud (replace 'your_point_cloud.pcd' with the actual path)
#     point_cloud = CopyPointCloud(pcd)

#     # Get the points as a NumPy array
#     points = np.asarray(point_cloud.points)

#     # Calculate the centroid of the point cloud
#     centroid = np.mean(points, axis=0)

#     # Translate the point cloud to move the centroid to the origin (zero)
#     normalized_points = points - centroid

#     # Calculate the maximum distance from the origin to normalize the point cloud
#     max_distance = np.max(np.linalg.norm(normalized_points, axis=1))

#     # Scale the point cloud to normalize it
#     normalized_points /= max_distance

#     # Create a new point cloud with the normalized points
#     normalized_point_cloud = o3d.geometry.PointCloud()
#     normalized_point_cloud.points = o3d.utility.Vector3dVector(normalized_points)
#     normalized_point_cloud.colors = point_cloud.colors
#     normalized_point_cloud.normals = point_cloud.normals


#     return normalized_point_cloud

In [4]:
pcd = ReadCleanPointCloud('/usr/mvl2/lgsm2n/ULProj/pcd_dataset/S1/input_pcd/S1_trim.ply')
#RenderPointCloud(pcd)

In [5]:
print("Downsample the point cloud with a voxel of 0.005")
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
#o3d.visualization.draw_geometries([downpcd])
print(downpcd)

Downsample the point cloud with a voxel of 0.005
PointCloud with 5378 points.


### Normalizing point cloud (messes with evaluation i think)

In [6]:
import open3d as o3d
import numpy as np

# Load your point cloud (replace 'your_point_cloud.pcd' with the actual path)
point_cloud = CopyPointCloud(pcd)

# Get the points as a NumPy array
points = np.asarray(point_cloud.points)

# Calculate the centroid of the point cloud
centroid = np.mean(points, axis=0)

# Translate the point cloud to move the centroid to the origin (zero)
normalized_points = points - centroid

# Calculate the maximum distance from the origin to normalize the point cloud
max_distance = np.max(np.linalg.norm(normalized_points, axis=1))

# Scale the point cloud to normalize it
normalized_points /= max_distance

# Create a new point cloud with the normalized points
normalized_point_cloud = o3d.geometry.PointCloud()
normalized_point_cloud.points = o3d.utility.Vector3dVector(normalized_points)
normalized_point_cloud.colors = point_cloud.colors
normalized_point_cloud.normals = point_cloud.normals

# Visualize the normalized point cloud
#o3d.visualization.draw_geometries([normalized_point_cloud])


In [5]:
import open3d as o3d

# Load your point cloud (replace 'your_point_cloud.pcd' with the actual path)
point_cloud = CopyPointCloud(pcd) # nor normalizing now

# Get the Z-coordinates of all points in the point cloud
pts = point_cloud.points
pts = np.asarray(pts)
z_coordinates = pts[:, 2]

# Find the highest Z value
highest_z = z_coordinates.max()

# Find the lowest Z value
lowest_z = z_coordinates.min()

print("Highest Z value:", highest_z)
print("Lowest Z value:", lowest_z)


Highest Z value: -0.31336745619773865
Lowest Z value: -0.6464345455169678


In [7]:
# thresholding base

# Load your point cloud (replace 'your_point_cloud.pcd' with the actual path)
point_cloud = CopyPointCloud(pcd)

# Define the threshold Z value below which you want to remove points
threshold_z =lowest_z+ 0.2  # Replace with your desired threshold value

# Get the Z-coordinates of all points in the point cloud
pts = point_cloud.points
pts = np.asarray(pts)
z_coordinates = pts[:, 2]

# Create a binary mask for points below the threshold Z value
below_threshold_mask = z_coordinates > threshold_z

# Apply the mask to filter out points below the threshold Z value
filtered_point_cloud = point_cloud.select_by_index(np.where(below_threshold_mask)[0])
WritePointCloud('workflow_steps/removed_base.ply', filtered_point_cloud)
# Visualize the filtered point cloud
#o3d.visualization.draw_geometries([filtered_point_cloud])

In [8]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=7, init_params='k-means++',) # could play with covariance_type='diag'
pts = filtered_point_cloud.points
pts = np.array(pts)
print(pts.shape)
colors = filtered_point_cloud.colors
colors = np.array(colors)
# normals = filtered_point_cloud.normals
# normals = np.array(normals)
# print(normals.shape)
pts_colors = np.column_stack([pts, colors])
labels_gm = gm.fit_predict(pts)
print(labels_gm)

(751, 3)
[4 3 5 3 0 5 5 4 5 1 3 3 0 6 6 5 4 2 3 2 2 2 3 2 6 0 3 4 0 6 5 4 6 2 0 5 6
 0 3 5 2 6 4 3 4 4 3 1 1 5 2 1 2 4 2 1 5 3 4 3 4 5 4 3 2 4 3 3 5 2 3 3 4 3
 0 3 0 2 3 0 2 4 3 1 4 4 4 5 3 0 1 4 5 3 1 1 6 1 6 5 5 6 0 1 4 6 2 2 4 5 5
 3 2 6 2 3 2 1 5 3 2 5 5 2 2 5 4 1 3 3 4 6 6 6 1 3 1 4 2 5 1 5 6 6 2 3 6 3
 6 0 6 3 5 2 4 6 6 5 5 5 4 6 6 4 6 5 4 2 6 5 0 0 6 4 3 2 4 4 3 3 5 6 3 1 6
 3 6 1 3 5 3 5 3 6 1 1 2 6 5 5 5 3 1 6 3 3 6 0 2 2 1 6 4 2 1 1 3 6 3 0 2 1
 0 1 4 0 5 3 2 1 3 3 1 4 3 0 0 5 5 3 1 6 6 3 2 2 0 0 5 6 6 1 6 1 5 2 5 5 1
 1 6 2 3 3 2 3 3 5 4 4 5 2 2 0 6 4 3 4 6 2 6 5 1 2 6 1 6 2 0 3 4 1 1 1 3 6
 2 3 4 3 1 5 2 3 1 3 0 1 4 4 4 1 3 2 1 2 4 3 2 3 5 6 1 3 5 4 3 0 4 1 1 6 0
 6 1 0 6 0 3 2 5 3 0 6 2 0 3 5 3 1 4 0 2 3 3 3 4 2 3 3 5 3 3 0 4 4 5 2 5 6
 3 5 5 6 3 2 6 0 3 5 3 3 3 2 4 3 3 3 5 4 4 1 1 5 2 5 6 3 1 6 5 6 6 3 2 3 5
 6 6 4 3 4 4 3 2 3 5 6 5 2 2 0 4 3 2 2 5 3 0 6 3 3 1 4 0 5 1 3 6 4 3 5 3 3
 6 3 4 2 1 4 3 1 1 6 0 3 4 5 3 6 1 6 3 3 0 3 5 6 5 2 0 6 1 6 4 5 3 1 2 4 3
 2 6 4 0 5 5 6 4

In [13]:
# seeing the similarity of clusters
means = gm.means_
print(means[6])

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

vector1 = means[5]
vector2 = means[2]

similarity = cosine_similarity([vector1], [vector2])
print(similarity)

euclidean_distance = distance.euclidean(vector1, vector2)
print(euclidean_distance)

[ 0.13597883 -0.26719002 -0.37518671]
[[0.84693207]]
0.22392844615221905


### Heirarchical clustering method

In [14]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist, squareform

# Calculate pairwise distances using pdist
distances = pdist(means, metric='euclidean')

# Convert the pairwise distances to a square distance matrix using squareform
distance_matrix = squareform(distances)

# Specify the number of clusters (in this case, 3)
num_clusters = 3

# Compute the linkage matrix
linkage_matrix = linkage(distance_matrix, method='ward')

# Perform hierarchical clustering and assign nodes to clusters
cluster_labels = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')

# Print cluster assignments
for node, cluster in enumerate(cluster_labels):
    print(f"Node {node + 1} belongs to Cluster {cluster}")

heirarchical_labels = cluster_labels[labels_gm]
print(heirarchical_labels)

Node 1 belongs to Cluster 2
Node 2 belongs to Cluster 1
Node 3 belongs to Cluster 2
Node 4 belongs to Cluster 3
Node 5 belongs to Cluster 1
Node 6 belongs to Cluster 1
Node 7 belongs to Cluster 1
[1 3 1 3 2 1 1 1 1 1 3 3 2 1 1 1 1 2 3 2 2 2 3 2 1 2 3 1 2 1 1 1 1 2 2 1 1
 2 3 1 2 1 1 3 1 1 3 1 1 1 2 1 2 1 2 1 1 3 1 3 1 1 1 3 2 1 3 3 1 2 3 3 1 3
 2 3 2 2 3 2 2 1 3 1 1 1 1 1 3 2 1 1 1 3 1 1 1 1 1 1 1 1 2 1 1 1 2 2 1 1 1
 3 2 1 2 3 2 1 1 3 2 1 1 2 2 1 1 1 3 3 1 1 1 1 1 3 1 1 2 1 1 1 1 1 2 3 1 3
 1 2 1 3 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 1 1 3 2 1 1 3 3 1 1 3 1 1
 3 1 1 3 1 3 1 3 1 1 1 2 1 1 1 1 3 1 1 3 3 1 2 2 2 1 1 1 2 1 1 3 1 3 2 2 1
 2 1 1 2 1 3 2 1 3 3 1 1 3 2 2 1 1 3 1 1 1 3 2 2 2 2 1 1 1 1 1 1 1 2 1 1 1
 1 1 2 3 3 2 3 3 1 1 1 1 2 2 2 1 1 3 1 1 2 1 1 1 2 1 1 1 2 2 3 1 1 1 1 3 1
 2 3 1 3 1 1 2 3 1 3 2 1 1 1 1 1 3 2 1 2 1 3 2 3 1 1 1 3 1 1 3 2 1 1 1 1 2
 1 1 2 1 2 3 2 1 3 2 1 2 2 3 1 3 1 1 2 2 3 3 3 1 2 3 3 1 3 3 2 1 1 1 2 1 1
 3 1 1 1 3 2 1 2 3 1 3 3 3 2 1 3 3 3 1 1 1 1 1 1 2 1 1

/tmp/ipykernel_4107250/1883245739.py:15: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage_matrix = linkage(distance_matrix, method='ward')


# Normals Clustering

In [17]:
# Goal: find average normal of each cluster and cluster together to separate leaves
segments = segment_point_cloud(filtered_point_cloud, labels_gm)
print(segments)

def estimate_normals(point_cloud, search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)):
    """
    Estimate normals for the point cloud.

    :param point_cloud: Open3D point cloud object.
    :param search_param: Parameters for KDTree search in normal estimation.
    :return: Point cloud with estimated normals.
    """
    point_cloud.estimate_normals(search_param=search_param)
    point_cloud.orient_normals_consistent_tangent_plane(k=30)
    return point_cloud

def calculate_average_normal(point_cloud):
    """
    Calculate the average normal vector of the point cloud.

    :param point_cloud: Point cloud with normals.
    :return: Average normal vector.
    """
    normals = np.asarray(point_cloud.normals)
    average_normal = np.mean(normals, axis=0)
    return average_normal

# finding normals
normals = []
for segment in segments:
    segment = estimate_normals(segment)
    avg_normal = calculate_average_normal(segment)
    print(avg_normal)
    normals.append(avg_normal)

# Calculate pairwise distances using pdist
distances = pdist(normals, metric='cosine')

# Convert the pairwise distances to a square distance matrix using squareform
distance_matrix = squareform(distances)

# Specify the number of clusters (in this case, 3)
num_clusters = 2

# Compute the linkage matrix
linkage_matrix = linkage(distance_matrix, method='ward')

# Perform hierarchical clustering and assign nodes to clusters
cluster_labels = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')

# Print cluster assignments
for node, cluster in enumerate(cluster_labels):
    print(f"Node {node + 1} belongs to Cluster {cluster}")

heirarchical_labels = cluster_labels[labels_gm]
print(heirarchical_labels)

[PointCloud with 74 points., PointCloud with 98 points., PointCloud with 93 points., PointCloud with 160 points., PointCloud with 106 points., PointCloud with 108 points., PointCloud with 112 points.]
[-0.77499007  0.21066302  0.48204593]
[ 0.31062284 -0.53162633  0.77476794]
[-0.79121165  0.26892034  0.35221714]
[0.70130099 0.54785478 0.07509219]
[ 0.12977645 -0.97038545  0.17969814]
[0.42850571 0.14257463 0.85871096]
[ 0.24503982 -0.84711538  0.44767508]
Node 1 belongs to Cluster 1
Node 2 belongs to Cluster 2
Node 3 belongs to Cluster 1
Node 4 belongs to Cluster 2
Node 5 belongs to Cluster 2
Node 6 belongs to Cluster 2
Node 7 belongs to Cluster 2
[2 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2 1 2 1 1 1 2 1 2 1 2 2 1 2 2 2 2 1 1 2 2
 1 2 2 1 2 2 2 2 2 2 2 2 2 1 2 1 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2
 1 2 1 1 2 1 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 1 1 2 2 2
 2 1 2 1 2 1 2 2 2 1 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2 2
 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 

/tmp/ipykernel_4107250/3136372841.py:46: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage_matrix = linkage(distance_matrix, method='ward')


In [18]:
def segment_point_cloud(point_cloud, labels):
    rgb_values = [
        [0.89411765, 0.10196078, 0.10980392],
        [0.21568627, 0.49411765, 0.72156863],
        [0.30196078, 0.68627451, 0.29019608],
        [0.59607843, 0.30588235, 0.63921569],
        [1.0, 0.49803922, 0.0],
        [1.0, 1.0, 0.2],
        [0.65098039, 0.3372549, 0.15686275],
        [0.96862745, 0.50588235, 0.74901961],
        [0.6, 0.6, 0.6],
        [0.89411765, 0.10196078, 0.10980392],
        [0.89411765, 0.10196078, 0.10980392],
        [0.21568627, 0.49411765, 0.72156863],
        [0.30196078, 0.68627451, 0.29019608],
        [0.59607843, 0.30588235, 0.63921569],
        [1.0, 0.49803922, 0.0],
        [1.0, 1.0, 0.2],
        [0.65098039, 0.3372549, 0.15686275],
        [0.96862745, 0.50588235, 0.74901961],
        [0.6, 0.6, 0.6],
        [0.89411765, 0.10196078, 0.10980392]
    ]

    # Create an empty segmented point cloud
    segmented_pc = o3d.geometry.PointCloud()
    
    # Assign cluster labels to each point
    segmented_pc.points = o3d.utility.Vector3dVector(point_cloud.points)
    segmented_pc.colors = o3d.utility.Vector3dVector(np.zeros_like(point_cloud.colors))
    for i, label in enumerate(labels):
        value = rgb_values[label]
        segmented_pc.colors[i] = value
        #segmented_pc.colors[i] = point_cloud.colors[i] if label != -1 else [0, 0, 0]  # Set unclustered points to black
    
    return segmented_pc

# Segment the point cloud based on the labels and centroids
segmented_pc_gm = segment_point_cloud(filtered_point_cloud, heirarchical_labels) # usually labe
WritePointCloud('workflow_steps/heir_step.ply', segmented_pc_gm)
# Visualize the segmented point cloud
o3d.visualization.draw_geometries([segmented_pc_gm])

In [16]:
### seeing if similar normals allow for combining clusters
def segment_point_cloud(point_cloud, labels):
    clouds = []
    #segmented_cloud = o3d.geometry.PointCloud()

    # Convert the labels list to a NumPy array
    labels = np.asarray(labels)

    # Get unique labels
    unique_labels = np.unique(labels)

    # Iterate over unique labels and extract points belonging to each segment
    for label in unique_labels:
        mask = labels == label
        segment_points = np.asarray(point_cloud.points)[mask]
        segment_colors = np.asarray(point_cloud.colors)[mask]
        segment_cloud = o3d.geometry.PointCloud()
        segment_cloud.points = o3d.utility.Vector3dVector(segment_points)
        segment_cloud.colors = o3d.utility.Vector3dVector(segment_colors)
        clouds.append(segment_cloud)

    return clouds

In [15]:
segmented_pcd = segment_point_cloud(filtered_point_cloud, heirarchical_labels)
print(segmented_pcd)

[PointCloud with 124 points., PointCloud with 149 points., PointCloud with 234 points.]


### Metrics

In [16]:
import open3d as o3d
import numpy as np

def find_matching_and_non_matching_points(predicted_pc, ground_truth_pc, tolerance=0.001):
    """
    Find matching and non-matching points between a predicted point cloud and a ground truth point cloud.

    Args:
    - predicted_pc: Open3D PointCloud object representing the predicted point cloud.
    - ground_truth_pc: Open3D PointCloud object representing the ground truth point cloud.
    - tolerance: A distance tolerance threshold for considering points as matching (default is 0.001).

    Returns:
    - matching_points: Indices of matching points in the predicted point cloud.
    - non_matching_predicted_points: Indices of non-matching points in the predicted point cloud.
    - non_matching_ground_truth_points: Indices of non-matching points in the ground truth point cloud.
    """
    # Convert point clouds to NumPy arrays for efficient processing
    predicted_points = np.asarray(predicted_pc.points)
    ground_truth_points = np.asarray(ground_truth_pc.points)

    # Find matching points within the specified tolerance
    matching_points = []
    non_matching_predicted_points = []
    non_matching_ground_truth_points = []

    for i, pred_point in enumerate(predicted_points):
        closest_distance = np.min(np.linalg.norm(ground_truth_points - pred_point, axis=1))
        if closest_distance <= tolerance:
            matching_points.append(i)
        else:
            non_matching_predicted_points.append(i)

    # Find non-matching points in the ground truth point cloud
    for j, gt_point in enumerate(ground_truth_points):
        closest_distance = np.min(np.linalg.norm(predicted_points - gt_point, axis=1))
        if closest_distance > tolerance:
            non_matching_ground_truth_points.append(j)

    return matching_points, non_matching_predicted_points, non_matching_ground_truth_points

# Example usage:
# Load the predicted and ground truth point clouds
predicted_pc = CopyPointCloud(segmented_pcd[0])  #o3d.io.read_point_cloud("predicted_point_cloud.pcd")
#predicted_pc = o3d.io.read_point_cloud('/usr/mvl2/lgsm2n/ULProj/pcd_dataset/S1/S1_trim.ply')  #o3d.io.read_point_cloud("predicted_point_cloud.pcd")
# RenderPointCloud(predicted_pc)
ground_truth_pc = o3d.io.read_point_cloud("/usr/mvl2/lgsm2n/ULProj/pcd_dataset/S4/S4_trimmed_leaf1.ply")
# RenderPointCloud(ground_truth_pc)
ground_truth_pc_normalized = NormalizedPointCloud(ground_truth_pc)
# RenderPointCloud(ground_truth_pc_normalized)

# Find matching and non-matching points
matching, non_matching_pred, non_matching_gt = find_matching_and_non_matching_points(predicted_pc, ground_truth_pc)
print(len(matching))
# Visualize the results or perform further analysis as needed

NameError: name 'NormalizedPointCloud' is not defined

In [ ]:
def calculate_iou(matching, non_matching_pred, non_matching_gt):
    intersection = len(matching)
    union = len(non_matching_pred) + len(non_matching_gt) + intersection
    iou = intersection / union
    return iou

def calculate_precision(matching, non_matching_pred):
    true_positives = len(matching)
    false_positives = len(non_matching_pred)
    precision = true_positives / (true_positives + false_positives)
    return precision

def calculate_recall(matching, non_matching_gt):
    true_positives = len(matching)
    false_negatives = len(non_matching_gt)
    recall = true_positives / (true_positives + false_negatives)
    return recall


def calculate_f1_score(matching, non_matching_pred, non_matching_gt):
    precision = calculate_precision(matching, non_matching_pred)
    recall = calculate_recall(matching, non_matching_gt)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

# iou = calculate_iou(matching=matching, non_matching_pred=non_matching_pred, non_matching_gt=non_matching_gt)
# precision = calculate_iou(matching=matching, non_matching_pred=non_matching_pred)
# recall = calculate_iou(matching=matching, non_matching_gt=non_matching_gt)
# f1 = calculate_iou(matching=matching, non_matching_pred=non_matching_pred, non_matching_gt=non_matching_gt)

# print(f'The IOU score is {iou}')
# print(f'The precision is {precision}')
# print(f'The recall is {recall}')
# print(f'The F1 score is {f1}')
iou = calculate_iou(matching=matching, non_matching_pred=non_matching_pred, non_matching_gt=non_matching_gt)
precision = calculate_precision(matching, non_matching_pred)  # Pass matching and non_matching_pred here
recall = calculate_recall(matching, non_matching_gt)  # Pass matching and non_matching_gt here
f1 = calculate_f1_score(matching, non_matching_pred, non_matching_gt)  # Pass all three arguments here

print(f'The IOU score is {iou}')
print(f'The precision is {precision}')
print(f'The recall is {recall}')
print(f'The F1 score is {f1}')

ZeroDivisionError: float division by zero

In [ ]:
from evaluation import PointCloudMetricsCalculator

predicted_pc = CopyPointCloud(segmented_pcd[0])  #o3d.io.read_point_cloud("predicted_point_cloud.pcd")
ground_truth_pc = o3d.io.read_point_cloud("/usr/mvl2/lgsm2n/ULProj/pcd_dataset/S4/S4_trimmed_leaf1.ply")

# Create an instance of the PointCloudMetricsCalculator
calculator = PointCloudMetricsCalculator(predicted_pc, ground_truth_pc)

# Calculate all metrics
metrics = calculator.CalculateMetrics()

# Print the metrics
for metric_name, metric_value in metrics.items():
    print(f'{metric_name}: {metric_value}')

IOU: 0.8070953436807096
Precision: 0.9479166666666666
Recall: 0.8445475638051044
F1 Score: 0.8932515337423312


# Class Implementation

In [21]:
from segment_pcd import SegmentLeavesPCD
from evaluation import PointCloudMetricsCalculator

seg_class = SegmentLeavesPCD()
ground_truth_pc = o3d.io.read_point_cloud("/usr/mvl2/lgsm2n/ULProj/pcd_dataset/S5/S5_trimmed_leaf1.ply")

segments = seg_class.SegmentLeaves('/usr/mvl2/lgsm2n/ULProj/pcd_dataset/S5/S5_trimmed.ply')

for segment in segments:
    # Create an instance of the PointCloudMetricsCalculator
    eval =  PointCloudMetricsCalculator(segment, ground_truth_pc)

    # Calculate all metrics
    metrics = eval.CalculateMetrics()

    # Print the metrics
    for metric_name, metric_value in metrics.items():
        print(f'{metric_name}: {metric_value}')

Creating Class


AttributeError: 'str' object has no attribute 'points'